# 1.3 Solve a fully-observed inverse problem

Given $x_T$, estimate $x_0$ by matching $G^{(T)}(x_0)$ to $x_T$. Use autodiff on $G$ to calculate gradients of an error metric w.r.t. $x_0$. Compare the resulting rollout to the original `true' simulation.

Compare three approaches:
- $argmin_{x_0} || x_T - G^{(T)}(x_i) ||$, i.e. $T$ steps in one go
- $argmin_{x_i} || x_{i+T_i} - G^{(T_i)}(x_i) ||$, i.e. $T_i$ steps at a time, with $\sum_i T_i = T$. In the extreme case of $T_i=1$, this becomes very similar to implicit numerical methods. Can invertible neural networks help beyond providing better initializations for $x_i$ ? 
- solving backwards: more of the extreme case of $\forall i: T_i=1$, however: Only for some forward numerical solvers can we just reverse time [1] and expect to return to initial conditions. Leap-frog works, but e.g. forward-Euler time-reversed is backward-Euler. 

Generally, how do these approaches differ around \& beyond the horizon of predictability? Which solutions do they pick, and how easy is it to get uncertainties from them?

[1] https://scicomp.stackexchange.com/questions/32736/forward-and-backward-integration-cause-of-errors?noredirect=1&lq=1

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import L96sim

from L96_emulator.util import dtype, dtype_np, device

res_dir = '/gpfs/work/nonnenma/results/emulators/L96/'
data_dir = '/gpfs/work/nonnenma/data/emulators/L96/'

### pick a (trained) emulator

In [ ]:
from L96_emulator.run import setup

exp_id = 24
if exp_id is None: 
    # loading 'perfect' (up to machine-precision-level quirks) L96 model in pytorch    
    conf_exp = '00_analyticalMinimalConvNet'
else:
    exp_names = os.listdir('experiments/')   
    conf_exp = exp_names[np.where(np.array([name[:2] for name in exp_names])==str(exp_id))[0][0]][:-4]

    args = setup(conf_exp=f'experiments/{conf_exp}.yml')
    args.pop('conf_exp')
    
print('conf_exp', conf_exp)


model_forwarder_str = 'rk4_default'

optimizer_str = 'LBFGS' #'LBFGS', 'SGD'
obs_operator_str = 'ObsOp_identity' #'ObsOp_subsampleGaussian'

## plot and compare results

In [ ]:
if not obs_operator_str is None:
    fn = f'fullyobs_initstate_tests_exp{exp_id}_{model_forwarder_str}_{optimizer_str}_{obs_operator_str}.npy'
else:
    fn = f'fullyobs_initstate_tests_exp{exp_id}_{model_forwarder_str}_{optimizer_str}.npy'

res = np.load(res_dir + 'results/data_assimilation/'+ fn, allow_pickle=True)[()]

loss_vals_LBFGS_full_backsolve=res['loss_vals_LBFGS_full_backsolve']
loss_vals_LBFGS_full_persistence=res['loss_vals_LBFGS_full_persistence']
loss_vals_LBFGS_full_chunks=res['loss_vals_LBFGS_full_chunks']
loss_vals_LBFGS_chunks=res['loss_vals_LBFGS_chunks']
loss_vals_LBFGS_recurse_chunks=res['loss_vals_LBFGS_recurse_chunks']

time_vals_LBFGS_full_backsolve=res['time_vals_LBFGS_full_backsolve']
time_vals_LBFGS_full_persistence=res['time_vals_LBFGS_full_persistence']
time_vals_LBFGS_full_chunks=res['time_vals_LBFGS_full_chunks']
time_vals_LBFGS_chunks=res['time_vals_LBFGS_chunks']
time_vals_backsolve=res['time_vals_backsolve']
time_vals_LBFGS_recurse_chunks=res['time_vals_LBFGS_recurse_chunks']

state_mses_LBFGS_full_backsolve=res['state_mses_LBFGS_full_backsolve']
state_mses_LBFGS_full_persistence=res['state_mses_LBFGS_full_persistence']
state_mses_LBFGS_full_chunks=res['state_mses_LBFGS_full_chunks']
state_mses_LBFGS_chunks=res['state_mses_LBFGS_chunks']
state_mses_backsolve=res['state_mses_backsolve']
state_mses_LBFGS_recurse_chunks=res['state_mses_LBFGS_recurse_chunks']

x_sols_LBFGS_full_backsolve=res['x_sols_LBFGS_full_backsolve'] 
x_sols_LBFGS_full_persistence=res['x_sols_LBFGS_full_persistence']
x_sols_LBFGS_full_chunks=res['x_sols_LBFGS_full_chunks']
x_sols_LBFGS_chunks=res['x_sols_LBFGS_chunks']
x_sols_backsolve=res['x_sols_backsolve']
x_sols_LBFGS_recurse_chunks=res['x_sols_LBFGS_recurse_chunks']

targets=res['targets']
initial_states=res['initial_states']

J = res['J']
n_steps = res['n_steps']
n_chunks = res['n_chunks']
n_chunks_recursive = res['n_chunks_recursive']
T_rollout = res['T_rollout']
dt = res['dt']
back_solve_dt_fac=res['back_solve_dt_fac']
n_starts = res['n_starts']

optimiziation_schemes = res['optimiziation_schemes']

In [ ]:
appr_names = {
    'LBFGS_chunks' : 'optim over single chunk (current chunk error)',
    'LBFGS_full_backsolve' : 'full optim, init from backsolve', 
    'LBFGS_full_chunks' : 'full optim, init from chunks',
    'LBFGS_full_persistence' : 'full optim, init from persistence',
    'LBFGS_recurse_chunks' : 'full optim, recursive from backsolve',
    'backsolve' : 'forward solve in reverse'
}

plt_styles = {
    'LBFGS_chunks' : 'r--',
    'LBFGS_full_backsolve' : 'b-', 
    'LBFGS_full_chunks' : 'm-',
    'LBFGS_full_persistence' : 'g-',
    'LBFGS_recurse_chunks' : 'k-',
    'backsolve' : 'c-'        
}

all_appr_names, all_losses, all_times, all_mses, all_plt_styles = [], [], [], [], []
for scheme_str in list(optimiziation_schemes.keys()):
    if optimiziation_schemes[scheme_str]:

        all_appr_names.append(appr_names[scheme_str])
        all_losses.append(res['loss_vals_'+scheme_str])
        all_times.append(res['time_vals_'+scheme_str])
        all_mses.append(res['state_mses_'+scheme_str])
        all_plt_styles.append(plt_styles[scheme_str])
"""
if all_losses[0].ndim ==2 :
    all_losses = [np.nanmean(l,axis=1) for l in all_losses]
if all_mses[0].ndim ==2 :
    all_mses = [np.nanmean(l,axis=1) for l in all_mses]
if all_times[0].ndim ==2 :
    all_times = [np.nanmean(l,axis=1) for l in all_times]
"""

In [ ]:
plt.figure(figsize=(16,20))
plt.subplot(3,1,1)
for i,loss in enumerate(all_losses):
    xx = np.linspace(0, T_rollout, loss.shape[0])
    plt.semilogy(xx, loss[:,0], all_plt_styles[i], label=all_appr_names[i])        
for i,loss in enumerate(all_losses):
    xx = np.linspace(0, T_rollout, loss.shape[0])
    plt.semilogy(xx, loss[:,1:], all_plt_styles[i])        

plt.legend()
plt.xlabel('rollout step')
plt.ylabel('neg. log-likelihood')
plt.title('loss during optimization for different initialization methods')


from L96_emulator.util import sortL96fromChannels, sortL96intoChannels

plt.subplot(3,1,2)
for i,mse in enumerate(all_mses):
    xx = np.linspace(0, T_rollout, mse.shape[0]+1)[1:]
    plt.semilogy(xx, mse[:,0], all_plt_styles[i], marker='o',label=all_appr_names[i])        
for i,mse in enumerate(all_mses):
    xx = np.linspace(0, T_rollout, mse.shape[0]+1)[1:]
    plt.semilogy(xx, mse[:,1:], all_plt_styles[i], marker='o')        
plt.legend()
plt.xlabel('rollout steps')
plt.ylabel('initial state MSE')
plt.title('initial state error for different initialization methods')

plt.subplot(3,1,3)
for i, tms in enumerate(all_times):
    tms[np.where(tms > 1e6)[0]] = np.nan
    xx = np.linspace(1, T_rollout, tms.shape[0])
    plt.semilogy(xx, tms[:,0], all_plt_styles[i], label=all_appr_names[i])        
for i, tms in enumerate(all_times):
    tms[np.where(tms > 1e6)[0]] = np.nan
    xx = np.linspace(1, T_rollout, tms.shape[0])
    plt.semilogy(xx, tms[:,1], all_plt_styles[i])        

plt.legend()
plt.xlabel('rollout steps')
plt.ylabel('computation time [s]')
plt.title('full computation time for different initialization methods')

plt.suptitle('exp_id : ' + conf_exp)

plt.show()

## compare against backsolve

In [ ]:
"""
import time
from L96_emulator.eval import Rollout

if J > 0:
    def negfun(t, x):
        return - f2(x, F, h, b, c, dX_dt, K, J)
else:
    def negfun(t, x):
        return - f1(x, F, dX_dt, K)

n_steps, lr, weight_decay = 200, 1.0, 0.0

loss_vals_backsolve = np.zeros(n_steps)
time_vals_backsolve = time.time() * np.ones(n_steps)

target = out[n_starts+T_rollout]
state_mses_backsolve = np.zeros(n_chunks)

x_init = np.zeros((len(n_starts), K*(J+1)))

i_ = 0

plt.figure(figsize=(12, 4))
for c_, dt_fac in enumerate([1, 10, 100, 1000]):
    plt.subplot(1,4,c_+1)
    for j in range(n_chunks):

        T_i = (j+1)*T_rollout//n_chunks
        times = np.linspace(0, dt*T_i, dt_fac*T_i+1)
        print('backward solving')
        for i__ in range(len(n_starts)):
            out2 = rk4_default(fun=negfun, y0=out[n_starts[i__]+T_rollout].copy(), times=times)
            x_init[i__] = out2[-1].copy()
        state_mses_backsolve[j] = ((x_init - target)**2).mean()
    plt.plot(T_rollout//n_chunks*np.arange(1,n_chunks+1), state_mses_backsolve)
    plt.xlabel('T_rollout')
    plt.ylabel('MSE of iniitial state estimate')
    plt.title(f'dt={dt/dt_fac}')
plt.show()
"""

### compare with plain gradient descent (SGD with single data point)

In [ ]:
"""

import time
from L96_emulator.eval import Rollout


n_starts = np.array([5000, 10000, 15000])
T_rollout, N = 100, len(n_starts)
n_chunks = 20

target = torch.as_tensor(out[n_starts+T_rollout], dtype=dtype, device=device)

x_init = out[n_starts+T_rollout].copy()
roller_outer_SGD = Rollout(model_forward, prediction_task='state', K=K, J=J, N=N, x_init=x_init)
x_init = roller_outer_SGD.X.detach().cpu().numpy().copy()

n_steps, lr, weight_decay = 500, 0.01, 0.0
roller_outer_SGD.train()

optimizer = torch.optim.Adam(roller_outer_SGD.parameters(), lr=lr, weight_decay=weight_decay)

#optimizer = torch.optim.LBFGS(params=roller_outer.parameters(), 
#                              lr=lr, 
#                              max_iter=20, 
#                              max_eval=None, 
#                              tolerance_grad=1e-07, 
#                              tolerance_change=1e-09, 
#                              history_size=100, 
#                              line_search_fn=None)
loss_vals_SGD = np.zeros(n_steps)
time_vals_SGD = time.time() * np.ones(n_steps)
for i in range(n_steps):
        optimizer.zero_grad()
        loss = ((roller_outer_SGD.forward(T=T_rollout) - target)**2).mean()
        loss.backward()
        optimizer.step()
        loss_vals_SGD[i] = loss.detach().cpu().numpy()
        time_vals_SGD[i] = time.time() - time_vals_SGD[i]
        print((time_vals_SGD[i], loss_vals_SGD[i]))
        
plt.figure(figsize=(8,2))
plt.semilogy(loss_vals_SGD, label='initialization')
plt.title('rollout final state loss across gradient descent steps')
plt.ylabel('MSE)')
plt.xlabel('gradient step')
plt.show()

"""

In [ ]:
plt.plot(tms[:,1])

# share notebook results via html file

In [ ]:
!jupyter nbconvert --output-dir='/gpfs/home/nonnenma/projects/lab_coord/mdml_wiki/marcel/emulators' --to html data_assimilation.ipynb